<CENTER>
    <a href="http://opendata.atlas.cern/release/2020/documentation/notebooks/intro.html" class="icons"><img src="../../images/ATLASOD.gif" style="width:40%"></a>
</CENTER>

<CENTER><h1>Simple CVS to ROOT C++ notebook example</h1></CENTER>

In [1]:
%jsroot on

This is a very simple example of convertt a CSV file to ROOT format and use it to produce some plots

In [2]:
#include "Riostream.h"
#include "TString.h"
#include "TFile.h"
#include "TTree.h"
#include "TSystem.h"
#include <stdio.h>
#include <stdlib.h>

We download the CSV ATLAS Open Data file. This is the input of a web-app called Histogram analysers, for the 8 TeV release

In [3]:
//This two lines can be commented out. You just need it once, and in case the CVS file was not provided already
system("rm outreach.csv");
system("wget http://opendata.atlas.cern/visualisations/CrossFilter/outreach.csv");

//If the file is downloaded with the line above, you *need* to remove the first line of the outreach.csv

--2020-02-24 11:16:31--  http://opendata.atlas.cern/visualisations/CrossFilter/outreach.csv
Resolving opendata.atlas.cern (opendata.atlas.cern)... 188.184.64.13
Connecting to opendata.atlas.cern (opendata.atlas.cern)|188.184.64.13|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8802904 (8.4M) [text/csv]
Saving to: ‘outreach.csv’

     0K .......... .......... .......... .......... ..........  0% 6.18M 1s
    50K .......... .......... .......... .......... ..........  1% 7.10M 1s
   100K .......... .......... .......... .......... ..........  1% 7.87M 1s
   150K .......... .......... .......... .......... ..........  2% 10.2M 1s
   200K .......... .......... .......... .......... ..........  2% 8.73M 1s
   250K .......... .......... .......... .......... ..........  3% 7.21M 1s
   300K .......... .......... .......... .......... ..........  4% 8.43M 1s
   350K .......... .......... .......... .......... ..........  4% 5.57M 1s
   400K .......... .......... .....

  6350K .......... .......... .......... .......... .......... 74% 4.68M 0s
  6400K .......... .......... .......... .......... .......... 75% 78.1M 0s
  6450K .......... .......... .......... .......... .......... 75% 16.1M 0s
  6500K .......... .......... .......... .......... .......... 76% 8.28M 0s
  6550K .......... .......... .......... .......... .......... 76% 12.9M 0s
  6600K .......... .......... .......... .......... .......... 77% 11.3M 0s
  6650K .......... .......... .......... .......... .......... 77% 19.6M 0s
  6700K .......... .......... .......... .......... .......... 78% 10.3M 0s
  6750K .......... .......... .......... .......... .......... 79% 17.0M 0s
  6800K .......... .......... .......... .......... .......... 79% 11.9M 0s
  6850K .......... .......... .......... .......... .......... 80% 12.7M 0s
  6900K .......... .......... .......... .......... .......... 80% 10.7M 0s
  6950K .......... .......... .......... .......... .......... 81% 12.8M 0s
  7000K ....

Let's tell ROOT that the file is a CSV file, i.e. the values are separated by commmas

In [4]:
TString dir = gSystem->UnixPathName(__FILE__);
dir.ReplaceAll("outreach.C","");
dir.ReplaceAll("/./","/");

We define our new ROOT file where to put the information

In [5]:
TFile *f = new TFile("outreach.root","RECREATE");

The next cell is very important, it shows the name and type of varibles that the CSV file has so the new ROOT-tree can be filled with the correct information 

In [6]:
TTree *tree = new TTree("ntuple","data from csv file");
// The file inside has ---->   type,  Channel,  NJets,  MET,  Mll,  LepDeltaPhi,  METLLDeltaPhi,  SumLepPt,  BTags,  weight
tree->ReadFile("outreach.csv","type/I:Channel/I:NJets/I:MET/F:Mll/F:LepDeltaPhi/F:METLLDeltaPhi/F:SumLepPt/F:BTags/F:weight/F",',');
f->Write();

Warning in <TTree::ReadStream>: Couldn't read formatted data in "type" for branch type on line 1; ignoring line
Warning in <TTree::ReadStream>: Read too few columns (1 < 10) in line 1; ignoring line


Please, notice that the line above generates a warning. This is because the first line of our CSV file contains the name of the colunms, but this is safely ignored by ROOT.

In [7]:
system("ls -lhrt outreach.*");

-rw-r--r-- 1 student student 8.4M Jan 15  2017 outreach.csv
-rw-r--r-- 1 student student 3.1M Feb 24 11:16 outreach.root


Notice also that in the output of the line above that the resulting ROOT files is ~37% of the size of the original SV file. This is another advantage of the ROOT format for this kind of datasets.

In [8]:
TFile *_file0 = TFile::Open("outreach.root");

Now, let's create some plots of the variables inside our new ROOT-tree

In [9]:
TCanvas *c3D = new TCanvas("c3D","c3D",10,10,400,400);
ntuple->Draw("MET:Mll:LepDeltaPhi","MET>0.");
c3D->Draw();

In [10]:
TCanvas *cz = new TCanvas("cz","cz",10,10,400,400);
ntuple->Draw("Mll:MET","weight>-999","colz");

In [11]:
cz->Draw();

In [12]:
TCanvas *c2D = new TCanvas("c2D","c2D",10,10,400,400);
ntuple->Draw("Mll:LepDeltaPhi","MET>0.","colz");
c2D->Draw();

Below you can try to create a more complex analysis, following what is done in the next notebooks examples in this repository.